# Modelling Canopy Gas-Exchange Process

<div class="alert alert-warning">
If figures/tables are not rendered correctly, make sure your notebook is `trusted` by choosing `Trust Notebook` command found in `View` - `Activate Command Palette` menu. Typing in the first letters (i.e. "tr") should give you the command.
</div>

In [ ]:
using Cropbox
using LeafGasExchange

We will use interactive plotting feature in this notebook. For this, we have to initialize [WebIO.jl](https://github.com/JuliaGizmos/WebIO.jl) manually when using [prebuilt system images](https://github.com/JuliaLang/PackageCompiler.jl) as we do with our JupyterHub environment.

In [ ]:
Cropbox.Interact.WebIO.setup(:ijulia)

In [ ]:
using Logging
# turn off warnings
Logging.disable_logging(Logging.Warn)

## Canopy Gas-Exchange Responses

For the following exercises, we're going to use gas-exchange model scaled to canopy level. Canopy-level models are named with suffix `C`. For example, `ModelC3BBC` is canopy-level C3 gas-exchange model coupled with Ball-Berry stomatal conductance model. We will be working with a rose canopy.

In [ ]:
using LeafGasExchange: C3, StomataBallBerry, Sun, Canopy, Radiation, LeafAngle, Weather

In [ ]:
# Create a gas-exchange configuration for a black cottownwood leaf (Populus trichocarpa)
# Vcm25, Jm25, g1 were calibrated for POTR
# other parameter estimates are from roses in Kim and Lieth (2003)
ge_potr = @config (
    :C3 => (
        Vcm25 = 103.1, Jm25 = 167.8, 
        Rd25 = 1.26, Tp25 = 11.55, Kc25 = 404, Ko25 = 248, Eac = 59.4, Eao = 36.0, Ear = 66.4, EaVc = 45.5,
        Eaj = 43.3, Hj = 219.4, Sj = 704.2, Γ25 = 36.9,
    ),
    :StomataBallBerry => (g0 = 0.01, g1 = 6.65), 
    :Weather => (
        PFD = 1500,
        CO2 = 400,
        RH = 60,
        T_air = 25,
        wind = 2.0,
    )
);

In [ ]:
ge_canopy = @config(
    ge_potr,
    Sun => (;
        day = 200,
        hour = 12,
    ),
    Canopy => (;
        LAI = 3,
    ),
    Radiation => (;
        leaf_angle_factor = 3,
        leaf_angle = LeafGasExchange.spherical,
    )
)

In [ ]:
co2_xstep = Weather => :CO2   => 10:10:1500;
pfd_xstep = Weather => :PFD   => 0:20:2000;
ta_xstep  = Weather => :T_air => 0:1:45;
rh_xstep  = Weather => :RH    => 0:1:100;

### Canopy photosynthetic response to light

In [ ]:
manipulate(ModelC3BBC, "weather.PFD", [:A_net, "sunlit_gasexchange.A_net", "shaded_gasexchange.A_net"];
    config = ge_canopy,
    parameters = (
        Canopy => :LAI => 0:0.1:20,
        Radiation => :leaf_angle => LeafAngle,
        Sun => :τ => 0:0.01:1, 
    ),
    xstep = pfd_xstep,
    ylim = (-10,50),
    kind = :line,
    xlab = "PFD",
    ylab = "A_net",
    names = ["Total", "Sunlit", "Shaded"],
)

### Canopy photosynthesis with respect to LAI

Here we test a hypothesis that a canopy with higher leaf area index (LAI) will have greater photosynthetic productivity. Discuss the modeling outcome and whether you accept or reject the hypothesis and why.

In [ ]:
manipulate(ModelC3BBC, :LAI, :A_net;
    config = ge_canopy,
    parameters = (
        Sun => :τ => 0:0.01:1,    
        Weather => :PFD  => 0:100:2500,
        Weather => :CO2  => 50:10:1500,
        Weather => :T_air  => 10:1:45,        
        C3 => :Vcm25 => 0:5.0:200, 
        C3 => :Jm25 =>  0:5.0:320, 
        C3 => :Rd25 =>  0:0.1:4.0,         
        Radiation => :leaf_angle => LeafAngle,        
    ),
    xstep = :0 => :LAI => 0:0.1:15,
    ylim = (-10,80),
    kind = :line,
)